# Optimal Path for Taking MTR Trains

In [55]:
import json
from pprint import PrettyPrinter, pprint
from itertools import zip_longest


In [56]:
weekdays_morningPeak = '../datasets/weakdays_morningPeak.json'
weekdays_eveningPeak = '../datasets/weekdays_eveningPeak.json'
weekdays_nonPeak = '../datasets/weekdays_nonPeak.json'

def load_graph(file_name):
    with open(file_name) as f:
        data = json.load(f)
        return data

dataset = load_graph(weekdays_morningPeak)

for item in dataset:
    print({item: dataset[item]})

{'tsuen wan/twl': [['tai wo hau/twl', 2.0]]}
{'tai wo hau/twl': [['tsuen wan/twl', 2.0], ['kwai hing/twl', 2.0]]}
{'kwai hing/twl': [['tai wo hau/twl', 2.0], ['kwai fong/twl', 2.0]]}
{'kwai fong/twl': [['kwai hing/twl', 2.0], ['lai king/twl', 2.0]]}
{'lai king/twl': [['kwai fong/twl', 2.0], ['mei foo/twl', 2.0], ['lai king/tcl', 0]]}
{'mei foo/twl': [['lai king/twl', 2.0], ['lai chi kok/twl', 2.0]]}
{'lai chi kok/twl': [['mei foo/twl', 2.0], ['cheung sha wan/twl', 2.0]]}
{'cheung sha wan/twl': [['lai chi kok/twl', 2.0], ['sham shui po/twl', 2.0]]}
{'sham shui po/twl': [['cheung sha wan/twl', 2.0], ['prince edward/twl', 2.0]]}
{'prince edward/twl': [['sham shui po/twl', 2.0], ['mong kok/twl', 2.0], ['prince edward/ktl', 0]]}
{'mong kok/twl': [['prince edward/twl', 2.0], ['yau ma tei/twl', 2.0], ['mong kok/ktl', 0]]}
{'yau ma tei/twl': [['mong kok/twl', 2.0], ['jordan/twl', 2.0], ['yau ma tei/ktl', 0]]}
{'jordan/twl': [['yau ma tei/twl', 2.0], ['tsim sha tsui/twl', 2.0]]}
{'tsim sha tsui

In [57]:
def graphSelector(period):
    global graph 
    if period == 0:
        graph = load_graph(weekdays_morningPeak)
    elif period == 1:
        graph = load_graph(weekdays_eveningPeak)
    elif period == 2:
        graph = load_graph(weekdays_nonPeak)

period = int(input('Please choose a time period: ( 0 - Morning hrs | 1 - Evening hrs | 2 - Non-peak hrs )'))
graphSelector(period)

for item in graph:
    print({item: graph[item]})

{'tsuen wan/twl': [['tai wo hau/twl', 2.0]]}
{'tai wo hau/twl': [['tsuen wan/twl', 2.0], ['kwai hing/twl', 2.0]]}
{'kwai hing/twl': [['tai wo hau/twl', 2.0], ['kwai fong/twl', 2.0]]}
{'kwai fong/twl': [['kwai hing/twl', 2.0], ['lai king/twl', 2.0]]}
{'lai king/twl': [['kwai fong/twl', 2.0], ['mei foo/twl', 2.0], ['lai king/tcl', 0]]}
{'mei foo/twl': [['lai king/twl', 2.0], ['lai chi kok/twl', 2.0]]}
{'lai chi kok/twl': [['mei foo/twl', 2.0], ['cheung sha wan/twl', 2.0]]}
{'cheung sha wan/twl': [['lai chi kok/twl', 2.0], ['sham shui po/twl', 2.0]]}
{'sham shui po/twl': [['cheung sha wan/twl', 2.0], ['prince edward/twl', 2.0]]}
{'prince edward/twl': [['sham shui po/twl', 2.0], ['mong kok/twl', 2.0], ['prince edward/ktl', 0]]}
{'mong kok/twl': [['prince edward/twl', 2.0], ['yau ma tei/twl', 2.0], ['mong kok/ktl', 0]]}
{'yau ma tei/twl': [['mong kok/twl', 2.0], ['jordan/twl', 2.0], ['yau ma tei/ktl', 0]]}
{'jordan/twl': [['yau ma tei/twl', 2.0], ['tsim sha tsui/twl', 2.0]]}
{'tsim sha tsui

In [58]:
def getNodeByStation(graph, station):
    for node in graph:
        if station == node.split('/')[0]:
            return node

getNodeByStation(dataset, 'lai king')

'lai king/twl'

In [59]:
# Dijkstra’s Shortest Path Algorithm
costs = dict()
routes = dict()

for station in graph:
    costs[station] = 999999
    routes[station] = {}

visited = set()
unvisited = set()

# Get Optimal Routes Function
def getOptimalRoutes(graph, costs, visited, unvisited, curr_station):

    if curr_station in unvisited:
        unvisited.remove(curr_station)
    visited.add(curr_station)

    for node in graph.items():
        station = node[0]
        adj_nodes = node[1]

        for adj_node in adj_nodes:
            adj_station = adj_node[0]
            cost = adj_node[1]

            if (station == curr_station and adj_station not in visited and costs[station] + cost < costs[adj_station]):

                unvisited.add(adj_station)   

                costs[adj_station] = costs[station] + cost
                 
                if curr_station.split('/')[1] != adj_station.split('/')[1]:
                    routes[station]['Line'] = routes[station]['Line'] + [adj_station.split('/')[1].upper()]
                else: routes[station]['Line']

                if curr_station.split('/')[1] != adj_station.split('/')[1]:
                    routes[station]['Transfer Station'] = routes[station]['Transfer Station'] + [adj_station.split('/')[0].title()]
                else: routes[station]['Transfer Station']

                routes[adj_station] = {
                    'Route': (
                        routes[station]['Route'] + ' -> ' + adj_station.split('/')[0].title()
                        if curr_station.split('/')[0] != adj_station.split('/')[0] 
                        else routes[station]['Route']
                    ),
                    'Line': routes[station]['Line'],
                    'Transfer Station': routes[station]['Transfer Station'],
                    'Weight': round(costs[adj_station])
                }
    
    costs[curr_station] = 999999
    optimal = min(costs, key = costs.get)

    if optimal not in visited:
        getOptimalRoutes(graph, costs, visited, unvisited, optimal)

In [60]:
start = input("Enter the Start Node: ")
goal = input("Enter the Goal Node: ")

start_node = getNodeByStation(dataset, start)
goal_node = getNodeByStation(dataset, goal)

unvisited.add(start_node)
routes[start_node] = {'Route': start_node.split('/')[0].title(), 'Line': [start_node.split('/')[1].upper()], 'Transfer Station': []}
costs[start_node] = 0

In [61]:
getOptimalRoutes(graph, costs, visited, unvisited, start_node)
print("Result:", routes[goal_node])

Result: {'Route': 'Lai King -> Nam Cheong -> Olympic -> Kowloon -> Hong Kong -> Central -> Admiralty -> Wan Chai -> Causeway Bay -> Tin Hau', 'Line': ['TWL', 'TCL', 'IL'], 'Transfer Station': ['Lai King', 'Central'], 'Weight': 21}


In [62]:
for result in routes[goal_node].items():
    if result[0] == 'Route':
        route = result[1]

    if result[0] == 'Line':
        line = result[1]

    if result[0] == 'Transfer Station':
        transfer_station = result[1]

    if result[0] == 'Weight':
        weight = result[1]

# Final Output
interchange = [[x for x in t if x is not None] for t in zip_longest(line, transfer_station)]
solution = '(' + str(start.title()) + ')'
for l in interchange: 
    if len(l) == 1:
        solution = solution + '--' + l[0] + '--'
    else:
        solution = solution + '--' + l[0] + '--' + '(' + l[1] + ')'
solution = solution + '(' + str(goal.title()) + ')'

print('~' + str(weight) + ' min(s)\t' + str(len(transfer_station)) + ' Interchange\t' + str(len(route.split(' -> ')))  + ' Stops')
print(solution)

~21 min(s)	2 Interchange	10 Stops
(Lai King)--TWL--(Lai King)--TCL--(Central)--IL--(Tin Hau)
